## Step 0: Importing the Appropriate Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import astropy
import emcee
import corner
import os
import glob

import astropy.constants as ac
from astropy import units as u
from astropy.convolution import Gaussian1DKernel, convolve
from astropy.modeling import models
from astropy.visualization import quantity_support
from scipy.optimize import minimize
from scipy import interpolate

## Step 1: Importing Data and Changing the Units of the Variables

We'll first import and read the following data:

1. chen+07_slshlh.dat; This contains the Spitzer IRS spectrum of BetaPic from ~5-35 microns. 

2. betapic_fluxes.dat; This contains assorted photometry from the optical and infrared with BetaPic. 

3. betapic_lws_cnt3_.txt; This contains the ISO LWS spectrum of BetaPic, from ~43 to 160 microns.

### 1.1: Using the correct folders to pull the data from and to put the data in

In [2]:
# to change to main directory
os.chdir('..')
cwd = os.getcwd()

# then the appropriate folders inside this main directory
Photom_IRS = cwd + "\\data\\betapic_data\DiskModels\Debris\BetaPic" # where the photometry and IRS Spectrum is found
LWS = cwd + "\\data\\betapic_data\DiskModels\Debris\BetaPic\iso\hpdp_62003530_3" # where the LWS Spectrum is found

Ice = cwd + "\\data\\water_ice_opacities_lsbrp" # where the different opacity files are found
Silicates = cwd + "\\data\\silicates_for_betapic" # where the different silicate files are found

### 1.2: Importing the Data 

In [3]:
# Changing directory to the photometry and IRS Spectrum files
os.chdir(Photom_IRS)

# Columns are all labelled (wavelength in microns, flux (F_nu) and uncertainty in Janskys, (Jy)).
Spitzer_IRS = np.loadtxt('chen+07_slshlh.dat')
Wavelength_IRS, Flux_IRS_Jy, Error_IRS_Jy = Spitzer_IRS[:,0], Spitzer_IRS[:,1], Spitzer_IRS[:,2]

# Columns are formatted as wavelength (mircons), flux (F_nu, Jy), uncertainty (Jy).
Photom = np.loadtxt('betapic_fluxes.dat')
Wavelength_Photom, Flux_Photom_Jy, Error_Photom_Jy = Photom[:,0], Photom[:,1], Photom[:,2]


# Changing directory to the LWS Spectrum file
os.chdir(LWS)

# Columns are formatted  as wavelength (microns), nu*F_nu (erg/cm^2/s), and uncertainty in nu*F_nu (same units).
ISO_LWS = np.loadtxt('betapic_lws_cnt3_.txt')
Wavelength_LWS, Flux_LWS, Error_LWS = ISO_LWS[:,0], ISO_LWS[:,1], ISO_LWS[:,2]

### 1.3: Rewriting the units
We need to convert the fluxes and error from Jansky to erg cm$^{-2}$ s$^{-1}$ . We know that 1 Jy equals to 10$^{23}$ erg cm$^{-2}$ s$^{-1}$ Hz$^{-1}$.

Such that we get: erg cm$^{-2}$ s$^{-1}$ = 10$^{-23}$ Hz Jy

In [4]:
# We can now calculate the flux using F = v*F_v (also making sure that the wavelength is in meters and not in microns)
Flux_IRS = 10**(-23)*Flux_IRS_Jy*(ac.c.value/(Wavelength_IRS*10**(-6)))
Flux_Photom = 10**(-23)*Flux_Photom_Jy*(ac.c.value/(Wavelength_Photom*10**(-6)))

# We can now also calculate the error in the fluxes
Error_IRS = 10**(-23)*Error_IRS_Jy*(ac.c.value/(Wavelength_IRS*10**(-6)))
Error_Photom = 10**(-23)*Error_Photom_Jy*(ac.c.value/(Wavelength_Photom*10**(-6)))

In [5]:
# Making sure no flux is zero/negative
Flux_IRS, Wavelength_IRS, Error_IRS = Flux_IRS[Flux_IRS>0], Wavelength_IRS[Flux_IRS>0], Error_IRS[Flux_IRS>0]
Flux_Photom, Wavelength_Photom, Error_Photom = Flux_Photom[Flux_Photom>0], Wavelength_Photom[Flux_Photom>0], Error_Photom[Flux_Photom>0]

# Removing the negative values
Flux_LWS, Wavelength_LWS, Error_LWS = Flux_LWS[Flux_LWS>0], Wavelength_LWS[Flux_LWS>0], Error_LWS[Flux_LWS>0]

# Removing the error equal to 0 from the photometry
Error_Photom, Flux_Photom, Wavelength_Photom = Error_Photom[Error_Photom!=0], Flux_Photom[Error_Photom!=0], Wavelength_Photom[Error_Photom!=0]

We need to convolve LWS Spectrum to smoothen it

In [6]:
# Create kernel & convolving lWS spectrum
g = Gaussian1DKernel(stddev=3)
z = convolve(Flux_LWS, g)

# Removing the tail of the LWS spectrum [by trial and error]
Wavelength_z, z, Error_z = Wavelength_LWS[6:], z[6:], Error_LWS[6:]

In [7]:
# Creating a list with all ice opacity file names
os.chdir(Ice)
ice = sorted(glob.glob("h2oice.p3p5.amin0p005.amax*.wb08c146"))

# Columns have the format: wavelength (microns), total opacity, albedo, absorption coefficient, asymmetry parameter g
# Read the files in with np.loadtxt('filename')

# Creating a list with all silicate opacity file names
os.chdir(Silicates)
silicates = sorted(glob.glob("sil.p3p5.amax*.extinc"))

# Returning to main directory
os.chdir(cwd)

## Chi Squared Fit

### Reading in the Ice and Silicate Files

In [8]:
def tau(kappa, mass, d):
    return kappa*(mass*(ac.M_earth).value*10**(3))/(d**2)

def chi_sq(model, flux, error):
    """Calculates the chi squared value."""
    return np.sum(((model-flux)/error)**2)

def spectrum_ice_si_tworing(wave, x_b, x0, kappa_ice, kappa_si, kappa_si2, d):
    """Creates a model for the flux of a star, for which a blackbody approximation is used,
    in combination with a ring for which both ice and silicates opacities are
    taken into account."""
    T, SolidAngle = x_b
    T_2, mass_i, mass_s, T_3, mass_s2 = x0
    
    bb = models.BlackBody(temperature=T*u.K)
    B_q_1 = bb(wave*u.micron)
    bb_2 = models.BlackBody(temperature=T_2*u.K)
    B_q_2 = bb_2(wave*u.micron)
    bb_3 = models.BlackBody(temperature=T_3*u.K)
    B_q_3 = bb_3(wave*u.micron)
    
    t_ice = tau(kappa_ice, mass_i*10**(-4), d)/0.0005
    t_si = tau(kappa_si, mass_s*10**(-2), d)/0.004
    
    t_si2 = tau(kappa_si2, mass_s2, d)/0.004
        
    F_1 = (SolidAngle)*B_q_1*(ac.c/(wave*u.micron))
    F_2 = B_q_2*(ac.c/(wave*u.micron))
    F_3 = B_q_3*(ac.c/(wave*u.micron))
    
    F = F_1 + (t_ice + t_si)*F_2 + t_si2*F_3
    return F.to(u.erg * u.cm**(-2) * u.s**(-1) * u.sr**(-1)).value
    

def chi_squared_ice_si_tworing(x0, x0_beta, wave_IRS, wave_LWS, flux_IRS, error_IRS, flux_LWS,
                               error_LWS, d, ice, silicate_1, silicate_2, weight):
    """Calculates the reduced chi^2 value for a given set of parameters and data points
    (with the corresponding errors) for a certain model. This definition is for the 
    the determination of the reduced chi^2 value of the flux of a spectrum."""
    for i in x0:
        if i <= 0:
            return np.inf
    
    f_ice = interpolate.interp1d(ice[:,0], ice[:,1]) #Interpolating to make the array's match
    f_si1 = interpolate.interp1d(silicate_1[:,0], silicate_1[:,1]) #Interpolating to make the array's match
    f_si2 = interpolate.interp1d(silicate_2[:,0], silicate_2[:,1]) #Interpolating to make the array's match
    
    kappa_ice_IRS, kappa_si_IRS_1, kappa_si_IRS_2 = f_ice(wave_IRS), f_si1(wave_IRS), f_si2(wave_IRS)
    model_IRS = spectrum_ice_si_tworing(wave_IRS, x0_beta, x0, kappa_ice_IRS, kappa_si_IRS_1, kappa_si_IRS_2, d)
    
    kappa_ice_LWS, kappa_si_LWS_1, kappa_si_LWS_2 = f_ice(wave_LWS), f_si1(wave_LWS), f_si2(wave_LWS)
    model_LWS = spectrum_ice_si_tworing(wave_LWS, x0_beta, x0, kappa_ice_LWS, kappa_si_LWS_1, kappa_si_LWS_2, d)
    
    chi = chi_sq(model_IRS, flux_IRS, error_IRS)/(len(model_IRS)) + weight*chi_sq(model_LWS, flux_LWS, error_LWS)/(len(model_LWS))
    return chi

In [9]:
d_BetaPic = (19.76*u.pc).to(u.cm).value
x0_BetaPic = [8200, 1.2699*10**(-17)]

fit_mask_IRS = ( (Wavelength_IRS<=Wavelength_IRS[-1]) )
fit_mask_LWS = ( (Wavelength_z<=120) )
weight = (np.mean(Error_LWS)/np.mean(Error_IRS))**2

# Fixed files and values from the fit to the LWS spectrum
ice_file = 'h2oice.p3p5.amin0p005.amax15.g.ab0p0005.wb08c146'
silicates_file = 'sil.p3p5.amax1mm.g.ab0p004.extinc'
estimates_LWS = [77, 2.7, 1.3] #Temp, M_I [e-4], M_S [e-2]

# Reading in the fixed ice and silicate files
os.chdir(Ice)
ice_dust = np.loadtxt(ice_file,skiprows=1)

os.chdir(Silicates)
silicate_ice = np.loadtxt(silicates_file,skiprows=1)

estimates = []
chi = []

# phi is defined as [dust temperature, solid angle, l]
phi = [estimates_LWS[0], estimates_LWS[1], estimates_LWS[2], 200, 1e-5] 

for i in range(len(silicates)): # Creating a for loop to loop over all silicate files
    os.chdir(Silicates)
    silicate_i = np.loadtxt(silicates[i],skiprows=1)
        
    estimates_i = minimize(fun=chi_squared_ice_si_tworing, x0 = phi, args=(x0_BetaPic, Wavelength_IRS[fit_mask_IRS],
                                                                           Wavelength_z[fit_mask_LWS], Flux_IRS[fit_mask_IRS], 
                                                                           Error_IRS[fit_mask_IRS], z[fit_mask_LWS], 
                                                                           Error_z[fit_mask_LWS], d_BetaPic, ice_dust, 
                                                                           silicate_ice, silicate_i, weight), 
                          method = 'Nelder-Mead')

    chi.append(estimates_i.fun)
    estimates.append(estimates_i.x)
        
j_best = np.where(chi==np.min(chi))[0][0] # Finding which silicate file is best

print('index silicates:', j_best, " - ", silicates[j_best])
print('chi^2:', chi[j_best])
print('[T_1, M_I, M_S1, T_2, M_S2]:', estimates[j_best])

index silicates: 2  -  sil.p3p5.amax100.g.ab0p004.extinc
chi^2: 104.34872734459688
[T_1, M_I, M_S1, T_2, M_S2]: [7.68579590e+01 1.51090580e+00 1.35898568e+00 2.12448979e+02
 2.54436122e-05]
